In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [2]:
csl_df = pd.read_csv('../../data/csl/Sites/CSL_he_PI_s46.csv', index_col=0)
X = csl_df.drop(['high_EBL','Sitenum'], axis=1, inplace=False)
y = csl_df['high_EBL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=7)
X_train                                                    

,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,...,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI
MomID,,,,,,,,,,,,,,,,,,,,,
46-00490,0,0,8,8,8,99.0,79,0.0,1,120.0,...,0,0,0,0,0,0,35,20,1,0
46-03267,0,0,8,8,8,99.0,88,0.0,1,122.0,...,0,0,0,0,0,0,29,33,1,1
46-05746,0,0,8,8,8,99.0,63,0.0,1,130.0,...,0,0,0,0,0,0,22,24,1,1
46-04895,0,0,8,8,8,99.0,91,0.0,1,152.0,...,0,0,0,0,0,0,25,30,1,1
46-06459,0,0,8,8,8,99.0,74,0.0,1,127.0,...,0,0,0,0,0,0,21,24,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46-00421,0,0,8,8,8,2.0,69,0.0,1,121.0,...,0,0,0,0,0,0,22,31,1,1
46-01335,0,0,8,8,8,99.0,85,0.0,1,136.0,...,0,0,0,0,0,0,23,31,1,1
46-04625,0,0,8,8,8,99.0,100,0.0,1,151.0,...,0,0,0,0,0,0,26,44,1,2


In [3]:
X.shape

(5805, 195)

<h1 align='center'>Exploratory Data Analysis</h1>

<h1 align='center'>Modelling</h1>

In [8]:
print(X_train.shape)
print(X.shape)
print(y.shape)

(4063, 195)
(5805, 195)
(5805,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [9]:
sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
#X_rus, y_rus = X_train, y_train

In [10]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(260, 195)
(260,)
[130 130]


<h1 align='center'>Basic Gradient Boosting</h1>

In [11]:
%%time
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
gb_rus.fit(X_rus, y_rus)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[1309  377]
 [   9   47]]
              precision    recall  f1-score   support

           0       0.99      0.78      0.87      1686
           1       0.11      0.84      0.20        56

    accuracy                           0.78      1742
   macro avg       0.55      0.81      0.53      1742
weighted avg       0.96      0.78      0.85      1742

ROC AUC prob1: 0.8567562701237079
MCC: 0.2530678007252819
F2: 0.36265432098765443
PR_AUC: 0.13549053779939826
CPU times: user 162 ms, sys: 2.07 ms, tota

In [12]:
%%time
# Not undersampled
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_weight = GradientBoostingClassifier()  
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
print(gb_weight.get_params())
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[1442  244]
 [  17   39]]
              precision    recall  f1-score   support

           0       0.99      0.86      0.92      1686
           1       0.14      0.70      0.23        56

    accuracy                           0.85      1742
   macro avg       0.56      0.78      0.57      1742
weighted avg       0.96      0.85      0.89      1742

ROC AUC prob1: 0.8497712252160652
MCC: 0.2638218096977305
F2: 0.3846153846153846
PR_AUC: 0.18852169889904058
CPU times: user 1.05 s, sys: 3.73 ms, total

<h1 align='center'>Hyperparameter Tuning with Cross Validation</h1>

In [13]:
%%time
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    max_depth = trial.suggest_int('max_depth',3,20)
    max_features = trial.suggest_int('max_features',10,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.08,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate) 
    return model

def objective(trial):
    model = create_model(trial)
    
    mcc_list = []
    roc_list = []
    pr_list = []
    f2_list = []
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train_c, X_test_c = X.iloc[train_idx], X.iloc[test_idx]
        y_train_c, y_test_c = y[train_idx], y[test_idx]
        sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
        X_res, y_res = sampler.fit_resample(X_train_c, y_train_c)

#        weights = class_weight.compute_sample_weight('balanced', y=y_train)
#        model.fit(X_train, y_train, sample_weight=weights)
        model.fit(X_res, y_res)
        y_pred = model.predict(X_test_c)
        probs = model.predict_proba(X_test_c)
        prob1 = probs[:, 1]  # Only positives
        precision, recall, pr_thresh = precision_recall_curve(y_test_c, prob1)
        roc = roc_auc_score(y_test_c, prob1)
        roc_list.append(roc)
        mcc = matthews_corrcoef(y_test_c, y_pred)  # MWB
        mcc_list.append(mcc)
        f2 = fbeta_score(y_test_c, y_pred, beta=2.0, average=None)[1]
        pr_auc = auc(recall, precision)
        pr_list.append(pr_auc)
        avg_precision = average_precision_score(y_test_c, prob1)
#        print(f'ROC: {roc}')
#        print(f'PR_AUC: {pr_auc}')
#        print(f'avg_prec: {avg_precision}')
        
    print(f'MCC: {mcc_list}')
    print(f'ROC: {roc_list}')
    print(f'PR_AUC: {pr_list}')
    return np.mean(mcc_list)

sampler = TPESampler(seed=7)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=25)
#study.optimize(objective,n_trials=5)

[I 2021-12-10 17:00:14,407] A new study created in memory with name: no-name-256eac24-d686-45cd-a14b-f54c07ede565
[I 2021-12-10 17:00:14,775] Trial 0 finished with value: 0.28638151657661026 and parameters: {'n_estimators': 24, 'subsample': 0.955983758448023, 'max_depth': 10, 'max_features': 17, 'learning_rate': 0.1973587414395923, 'min_samples_leaf': 27}. Best is trial 0 with value: 0.28638151657661026.


MCC: [0.30564699604649737, 0.3074230828968115, 0.2695662766728247, 0.26288971069030764]
ROC: [0.8878549254183388, 0.8728918458920006, 0.8537211821135695, 0.8154664484451718]
PR_AUC: [0.21174375648411567, 0.11391911428393207, 0.11943774453643656, 0.10796089462368812]


[I 2021-12-10 17:00:15,385] Trial 1 finished with value: 0.27750364589252063 and parameters: {'n_estimators': 105, 'subsample': 0.8144102266719524, 'max_depth': 7, 'max_features': 15, 'learning_rate': 0.16150759953451288, 'min_samples_leaf': 41}. Best is trial 0 with value: 0.28638151657661026.


MCC: [0.2773782408833501, 0.31280523508710295, 0.25501715853810003, 0.26481394906152955]
ROC: [0.8810176421594609, 0.8896178245396874, 0.849002011449791, 0.8234982117960842]
PR_AUC: [0.19859072731377486, 0.1360634724770983, 0.13353289660387305, 0.12231766446422083]


[I 2021-12-10 17:00:16,041] Trial 2 finished with value: 0.2800571664895577 and parameters: {'n_estimators': 82, 'subsample': 0.813187269381181, 'max_depth': 8, 'max_features': 20, 'learning_rate': 0.10560624242958987, 'min_samples_leaf': 23}. Best is trial 0 with value: 0.28638151657661026.


...

MCC: [0.3031999306678697, 0.3136711607209051, 0.26664142220073755, 0.2616206462664706]
ROC: [0.8896039978799122, 0.8854711434318427, 0.8518335138480582, 0.8145117294053463]
PR_AUC: [0.18453309775139434, 0.1323306240331567, 0.13736964808675753, 0.12405138731171858]


[I 2021-12-10 17:00:29,025] Trial 23 finished with value: 0.2983154329226311 and parameters: {'n_estimators': 10, 'subsample': 0.9397223442136705, 'max_depth': 20, 'max_features': 11, 'learning_rate': 0.18885759166233912, 'min_samples_leaf': 45}. Best is trial 17 with value: 0.2983154329226311.


MCC: [0.3148743629431107, 0.3123602063773523, 0.2890074724891535, 0.2770196898809078]
ROC: [0.8798440221094873, 0.9056165867244313, 0.8503171901593687, 0.8599063466084743]
PR_AUC: [0.13950454331950254, 0.18673299320683023, 0.1317132429892332, 0.11677633882152202]


[I 2021-12-10 17:00:29,451] Trial 24 finished with value: 0.2873656249449844 and parameters: {'n_estimators': 31, 'subsample': 0.9312220930123101, 'max_depth': 16, 'max_features': 12, 'learning_rate': 0.1866563240882886, 'min_samples_leaf': 46}. Best is trial 17 with value: 0.2983154329226311.


MCC: [0.29341404627307777, 0.31632653068859906, 0.27541746662263034, 0.26430445619563053]
ROC: [0.8924282577421065, 0.8837536747640415, 0.8516787869410491, 0.8555116081711825]
PR_AUC: [0.215738744105224, 0.12912321789301826, 0.1146087670953535, 0.14299037641886148]
CPU times: user 15 s, sys: 249 ms, total: 15.2 s
Wall time: 15 s


In [14]:
best = study.best_trial
best

FrozenTrial(number=17, values=[0.2983154329226311], datetime_start=datetime.datetime(2021, 12, 10, 17, 0, 25, 851732), datetime_complete=datetime.datetime(2021, 12, 10, 17, 0, 26, 179753), params={'n_estimators': 11, 'subsample': 0.9803712313231584, 'max_depth': 3, 'max_features': 10, 'learning_rate': 0.1832536562134998, 'min_samples_leaf': 50}, distributions={'n_estimators': IntUniformDistribution(high=200, low=10, step=1), 'subsample': UniformDistribution(high=1.0, low=0.8), 'max_depth': IntUniformDistribution(high=20, low=3, step=1), 'max_features': IntUniformDistribution(high=20, low=10, step=1), 'learning_rate': UniformDistribution(high=0.2, low=0.08), 'min_samples_leaf': IntUniformDistribution(high=50, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=17, state=TrialState.COMPLETE, value=None)

In [15]:
%%time
gb_params = study.best_params
#gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[1350  336]
 [   5   51]]
              precision    recall  f1-score   support

           0       1.00      0.80      0.89      1686
           1       0.13      0.91      0.23        56

    accuracy                           0.80      1742
   macro avg       0.56      0.86      0.56      1742
weighted avg       0.97      0.80      0.87      1742

ROC AUC prob1: 0.8833990001694627
MCC: 0.30187518867674695
F2: 0.4173486088379706
PR_AUC: 0.17095294177771772
CPU times: user 32 ms, sys: 1 ms, total: 33 ms
Wall time: 31.6 ms


In [16]:
%%time
gb_params = study.best_params
gb_params['random_state'] = 42
gb_weight = GradientBoostingClassifier(**gb_params)
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[1370  316]
 [  17   39]]
              precision    recall  f1-score   support

           0       0.99      0.81      0.89      1686
           1       0.11      0.70      0.19        56

    accuracy                           0.81      1742
   macro avg       0.55      0.75      0.54      1742
weighted avg       0.96      0.81      0.87      1742

ROC AUC prob1: 0.8421136248093544
MCC: 0.22288995029478373
F2: 0.3367875647668394
PR_AUC: 0.11925043631444184
CPU times: user 64.7 ms, sys: 5.04 ms, total: 69.7 ms
Wall time: 67.3 ms


In [17]:
# No Sitenum - 5
coeffs_wgt = pd.Series(data=gb_weight.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs_wgt[0:20]

Delmode           0.291215
spontlabor        0.195732
Parity            0.090745
TrialLabor        0.078018
Induction         0.047108
MthInd_Oxy        0.035118
HxnumCS           0.027300
prelaborCD        0.026633
new_high_BMI      0.024017
CS_FTP            0.023671
new_BMI           0.021933
Delfetalpos       0.013585
Dilat_lst         0.012538
AdmSBP            0.011563
new_age           0.010891
uscar             0.010769
Lac_None          0.010673
CS_NRFHT          0.010667
Admreason         0.010346
high_Gravidity    0.007625
dtype: float64

In [1]:
plt.rcParams["figure.figsize"] = (14, 8)
x_labels = coeffs_wgt.index.values[0:25]
print(x_labels)
fig, ax = plt.subplots(1,1)  # Create a figure and an axes.
ax.set_title('Top Predictors: All Sites combined; target: High_EBLoss, vars: PI')
ax.bar(x_labels, coeffs_wgt.values[0:25])
ax.set_ylabel('Coeff')
ax.set_xlabel('Variable')
plt.draw()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=16)
#plt.xlim(1,20)
plt.show()

NameError: name 'plt' is not defined

In [19]:
# No Sitenum - 5
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs[0:20]

Delmode           0.827671
spontlabor        0.097039
new_age           0.032086
Augment           0.012469
Lac_None          0.009450
BESTGA            0.006233
ROMmeth           0.005280
Admcontract       0.005178
high_Gravidity    0.003397
momrace_new       0.001198
Hxcsection        0.000000
Hxanemia          0.000000
Hxasthma          0.000000
HxGIdis           0.000000
Hostype           0.000000
HospElectInd      0.000000
Hxdepression      0.000000
HxnumCS           0.000000
Hxheartdis        0.000000
Hxmacrosomia      0.000000
dtype: float64

In [21]:
gb_params # 5 runs

{'n_estimators': 11,
 'subsample': 0.9803712313231584,
 'max_depth': 3,
 'max_features': 10,
 'learning_rate': 0.1832536562134998,
 'min_samples_leaf': 50,
 'random_state': 42}

In [2]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [ ]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

In [24]:
np.mean([0.3149689279176914, 0.31449457458602254, 0.31066798538596907, 0.3219519143488398])

0.3155208505596307

<h1 align='center'>Thanks and make sure to learn!</h1>